In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore')

### RegressorModule

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import make_regression
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
X_regr, y_regr = make_regression(1000,20,n_informative=10,random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32)/100
y_regr = y_regr.reshape(-1,1)

In [5]:
from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline

class RegressorModule(nn.Module):
    def __init__(self,num_units=10,nonlin=F.relu):
        super(RegressorModule,self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dense0 = nn.Linear(20,num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units,10)
        self.output = nn.Linear(10,1)
    def forward(self,X,**kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

nn_regressor = NeuralNetRegressor(RegressorModule,max_epochs=20,lr=0.1)
nn_regressor_pipeline = Pipeline([('net',nn_regressor)])

In [6]:
nn_regressor_pipeline.fit(X_regr,y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.6521        3.9025  0.1130
      2        4.2853        3.2140  0.0410
      3        2.4202        0.5702  0.0640
      4        0.4799        0.2762  0.0740
      5        0.2664        0.4691  0.0590
      6        0.3470        0.1310  0.0910
      7        0.1079        0.1244  0.0880
      8        0.0805        0.0744  0.0380
      9        0.0744        0.1681  0.0710
     10        0.1528        0.1326  0.0580
     11        0.1552        0.1704  0.0690
     12        0.1021        0.0694  0.0610
     13        0.0634        0.0530  0.0640
     14        0.0298        0.0342  0.0920
     15        0.0368        0.0375  0.0580
     16        0.0234        0.0326  0.0530
     17        0.0396        0.0398  0.0490
     18        0.0281        0.0368  0.0710
     19        0.0417        0.0334  0.0590
     20        0.0232        0.0294  0.0550


Pipeline(steps=[('net',
                 <class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
))])

In [7]:
y_pred = nn_regressor_pipeline.predict(X_regr[:5])
y_pred

array([[ 0.24164385],
       [-1.5176821 ],
       [-0.89105093],
       [-0.2990006 ],
       [-0.51240546]], dtype=float32)

### Save and load model

In [8]:
import pickle

file_name = 'nn_regressor.pkl'
with open(file_name,'wb') as pkl_file:
    pickle.dump(nn_regressor_pipeline,pkl_file)
with open(file_name,'rb') as saved_model:
    new_nn_regressor_pipeline = pickle.load(saved_model)

### save only model params

In [19]:
nn_regressor_pipeline.named_steps['net'].save_params(f_params=file_name)
new_nn_regressor = NeuralNetRegressor(RegressorModule,max_epochs=20,lr=0.1).initialize()
new_nn_regressor.load_params(file_name)